<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [2]:
import numpy as np
import pandas as pd
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [15]:
path = './module4-topic-modeling/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'

In [17]:
%pwd

'C:\\Users\\Emma\\Desktop\\DS-Unit-4-Sprint-1-NLP-master\\DS-Unit-4-Sprint-1-NLP-master\\module4-topic-modeling'

In [4]:
def gather_data(path_to_data):
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path, f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(str(text)))
    return data                           

In [20]:
df = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [21]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [22]:
df['brand'] = df['brand'].apply(lambda x: x.lower())
df['brand'].value_counts()

amazon          16153
amazonbasics    12179
Name: brand, dtype: int64

In [23]:
df.head(1)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."


In [26]:
df.shape, df.columns

((28332, 24),
 Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
        'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
        'manufacturerNumber', 'reviews.date', 'reviews.dateSeen',
        'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
        'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
        'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
       dtype='object'))

# Fit Genism LDA topic model to Amazon reviews

In [25]:
#first we gotta get the tokens
#cash money 

STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [29]:
#got em
df['tokens'] = df['reviews.text'].apply(tokenize)

In [30]:
#I am going to try the doc streaming with the yeild statement

def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strp('\n')
                    tokens = tokenize(text)
                yield tokens
              

In [38]:
streaming_data = doc_stream(df['tokens'])

In [39]:
next(streaming_data)

#it isn't working since it is in a dataframe, maybe I can save the entire thing as a list? 
#maybe this should stay a stretch goal 

TypeError: listdir: path should be string, bytes, os.PathLike or None, not Series

In [40]:
id2word = corpora.Dictionary(df['tokens'])

In [41]:
id2word.token2id['great']

22

In [42]:
id2word[22]

#yey it worked!

'great'

In [ ]:
#pshould I take out the stop words now?
STOPWORDS

#okay I forgot I took them out already, lets filter extreme values and create the corpus

In [48]:
len(id2word.keys())

9621

In [49]:
id2word.filter_extremes(no_below=5, no_above=0.95)

In [50]:
len(id2word.keys())

3581

In [51]:
#corpus

corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [52]:
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [53]:
#now we can fit the lda
#very excite

lda = LdaMulticore(corpus=corpus, #we gotta give them the bag of words
                  id2word=id2word, #we wanna see the words
                  random_state=42, 
                  num_topics = 10, #just gonna try ten, probably not a huge variety of reviews
                  passes= 25, #he said to amp it up to 100, but imma start small
                  workers=8)

#ok lets go

In [54]:
lda.print_topics()

#yey it worked

[(0,
  '0.057*"kids" + 0.040*"tablet" + 0.035*"games" + 0.025*"love" + 0.021*"great" + 0.020*"apps" + 0.017*"play" + 0.017*"year" + 0.015*"time" + 0.014*"old"'),
 (1,
  '0.164*"great" + 0.129*"price" + 0.127*"good" + 0.043*"product" + 0.030*"value" + 0.027*"quality" + 0.023*"deal" + 0.020*"buy" + 0.020*"works" + 0.018*"tablet"'),
 (2,
  '0.019*"tablet" + 0.018*"device" + 0.014*"little" + 0.012*"like" + 0.012*"use" + 0.012*"amazon" + 0.009*"screen" + 0.009*"charging" + 0.008*"slow" + 0.007*"case"'),
 (3,
  '0.108*"easy" + 0.086*"use" + 0.052*"love" + 0.027*"tablet" + 0.025*"great" + 0.024*"recommend" + 0.019*"best" + 0.015*"set" + 0.015*"product" + 0.014*"kindle"'),
 (4,
  '0.073*"kindle" + 0.019*"screen" + 0.019*"read" + 0.018*"light" + 0.018*"love" + 0.017*"new" + 0.015*"reading" + 0.013*"like" + 0.013*"better" + 0.011*"reader"'),
 (5,
  '0.048*"tablet" + 0.036*"amazon" + 0.033*"great" + 0.017*"apps" + 0.014*"works" + 0.014*"google" + 0.014*"prime" + 0.013*"play" + 0.013*"store" + 0.0

In [55]:
#lets make it look pretty

words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics()]


In [57]:
topics = [' '.join(t[0:5]) for t in words]    print(t, end="\n\n")


In [59]:
for id, t in enumerate(topics):
    print(f"_______________Topic {id}________________")
    print(t, end="\n\n")
    

#it seems okay, but as in the lesson we need to keep unpacking this 

_______________Topic 0________________
kids tablet games love great

_______________Topic 1________________
great price good product value

_______________Topic 2________________
tablet device little like use

_______________Topic 3________________
easy use love tablet great

_______________Topic 4________________
kindle screen read light love

_______________Topic 5________________
tablet amazon great apps works

_______________Topic 6________________
tablet books reading screen size

_______________Topic 7________________
batteries long amazon brand work

_______________Topic 8________________
work gift bought loved christmas

_______________Topic 9________________
loves old bought year tablet



# Create some dope visualization of the topics

In [60]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [61]:
pyLDAvis.gensim.prepare(lda, corpus, id2word) #this should give us the bubblemap

C:\Users\Emma\.conda\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.187989 -0.039902       1        1  22.917118
0     -0.196919  0.058169       2        1  11.703265
5      0.050748  0.075478       3        1  10.436888
4      0.009161  0.112129       4        1   9.838304
6      0.015059  0.142059       5        1   9.479208
1      0.219845 -0.079668       6        1   8.765888
2      0.004801  0.104020       7        1   7.666090
9     -0.238310 -0.142679       8        1   7.452909
3     -0.062441  0.037462       9        1   7.116559
8      0.010067 -0.267069      10        1   4.623763, topic_info=     Category         Freq       Term        Total  loglift  logprob
18    Default  7578.000000  batteries  7578.000000  30.0000  30.0000
17    Default  4917.000000      price  4917.000000  29.0000  29.0000
34    Default  5569.000000       good  5569.000000  28.0000  28.0000
21    Default  9015.000000      great  9015.000000  27.0000  27.0000
754   Default  3043.000000       easy  3043.000000  26.0000  26.0000
1856  Default  2219.000000      loves  2219.000000  25.0000  25.0000
9     Default  2233.000000       work  2233.000000  24.0000  24.0000
2075  Default  3046.000000     kindle  3046.000000  23.0000  23.0000
94    Default  4509.000000        use  4509.000000  22.0000  22.0000
520   Default  2139.000000        old  2139.000000  21.0000  21.0000
191   Default  2449.000000       kids  2449.000000  20.0000  20.0000
2432  Default  6997.000000     tablet  6997.000000  19.0000  19.0000
105   Default  1768.000000       year  1768.000000  18.0000  18.0000
28    Default  3234.000000     bought  3234.000000  17.0000  17.0000
884   Default  1152.000000       gift  1152.000000  16.0000  16.0000
90    Default  3499.000000       love  3499.000000  15.0000  15.0000
164   Default  2048.000000    product  2048.000000  14.0000  14.0000
651   Default  1712.000000      games  1712.000000  13.0000  13.0000
31    Default   930.000000  christmas   930.000000  12.0000  12.0000
386   Default  1003.000000      value  1003.000000  11.0000  11.0000
20    Default  1274.000000      brand  1274.000000  10.0000  10.0000
1267  Default   961.000000   daughter   961.000000   9.0000   9.0000
7     Default  1300.000000    quality  1300.000000   8.0000   8.0000
23    Default  2423.000000       long  2423.000000   7.0000   7.0000
945   Default  1195.000000    reading  1195.000000   6.0000   6.0000
431   Default   807.000000       deal   807.000000   5.0000   5.0000
2086  Default  1494.000000      books  1494.000000   4.0000   4.0000
727   Default   979.000000        son   979.000000   3.0000   3.0000
1547  Default  1845.000000       apps  1845.000000   2.0000   2.0000
900   Default   450.000000      loved   450.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
260   Topic10   259.061951       fine   535.866821   2.3471  -4.0047
9     Topic10   867.335999       work  2233.290771   2.1282  -2.7963
179   Topic10   165.673981         ok   318.970917   2.4189  -4.4517
220   Topic10   175.480133     worked   352.531738   2.3763  -4.3942
161   Topic10    93.034996      thank   169.658310   2.4732  -5.0288
1948  Topic10    33.770531        win    51.817249   2.6458  -6.0422
16    Topic10   299.252594      happy   849.744263   2.0303  -3.8605
1385  Topic10    73.581055   returned   141.149719   2.4225  -5.2634
290   Topic10    89.039291     couldn   191.281464   2.3093  -5.0727
156   Topic10    95.692200       gave   217.488235   2.2530  -5.0006
1891  Topic10    33.257843       xmas    52.470078   2.6180  -6.0575
1108  Topic10    54.210251     person   107.848442   2.3861  -5.5689
1596  Topic10    45.152611       told    83.386795   2.4605  -5.7517
109   Topic10    70.319008    arrived   165.035446   2.2208  -5.3087
28    Topic10   479.633972     bought  3234.063232   1.1655  -3.3887
248   Topic10   174.806488  excellent   699.265015   1.6876  -4.39

In [ ]:
#it seems that maybe this is too many topics, as there is a cluster of several
#bubbles all together

#also, should I add some of these common words to the stop list? for now I will
#leave them in, and ask about that at standup

### trying again with less topics

In [63]:
import warnings
warnings.filterwarnings("ignore")

In [69]:
lda = LdaMulticore(corpus=corpus, 
                  id2word=id2word, 
                  random_state=43, 
                  num_topics = 6, #I am gonna try six, because there were like four stuck together with 10 
                  passes= 25, #I would do the code that picks your topics, but it crashed after running 
                  workers=8)# through lunch

In [70]:
lda.print_topics()

[(0,
  '0.049*"kindle" + 0.023*"easy" + 0.022*"love" + 0.022*"read" + 0.021*"books" + 0.020*"reading" + 0.018*"use" + 0.017*"like" + 0.014*"light" + 0.013*"screen"'),
 (1,
  '0.023*"kindle" + 0.016*"bought" + 0.014*"charge" + 0.013*"tablet" + 0.012*"amazon" + 0.011*"buy" + 0.010*"device" + 0.009*"best" + 0.009*"new" + 0.008*"use"'),
 (2,
  '0.101*"batteries" + 0.028*"long" + 0.021*"work" + 0.019*"amazon" + 0.017*"brand" + 0.016*"good" + 0.016*"battery" + 0.015*"price" + 0.013*"great" + 0.012*"brands"'),
 (3,
  '0.127*"great" + 0.094*"good" + 0.088*"price" + 0.043*"product" + 0.027*"buy" + 0.026*"tablet" + 0.020*"value" + 0.018*"quality" + 0.016*"works" + 0.016*"deal"'),
 (4,
  '0.047*"tablet" + 0.026*"amazon" + 0.023*"great" + 0.018*"use" + 0.016*"apps" + 0.015*"screen" + 0.013*"good" + 0.010*"play" + 0.010*"like" + 0.009*"games"'),
 (5,
  '0.043*"tablet" + 0.036*"loves" + 0.031*"old" + 0.030*"kids" + 0.030*"bought" + 0.028*"year" + 0.025*"love" + 0.025*"use" + 0.023*"great" + 0.023*"e

In [71]:
words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics()]

In [72]:
topics = [' '.join(t[0:5]) for t in words]

In [73]:
for id, t in enumerate(topics):
    print(f"_______________Topic {id}________________")
    print(t, end="\n\n")

_______________Topic 0________________
kindle easy love read books

_______________Topic 1________________
kindle bought charge tablet amazon

_______________Topic 2________________
batteries long work amazon brand

_______________Topic 3________________
great good price product buy

_______________Topic 4________________
tablet amazon great use apps

_______________Topic 5________________
tablet loves old kids bought



In [74]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.201622  0.104930       1        1  24.570812
4      0.044778 -0.013413       2        1  19.567377
5      0.190613 -0.157903       3        1  19.373442
0      0.112047  0.078725       4        1  13.732844
3     -0.205638 -0.164893       5        1  12.141881
1      0.059823  0.152554       6        1  10.613634, topic_info=     Category         Freq        Term        Total  loglift  logprob
18    Default  7646.000000   batteries  7646.000000  30.0000  30.0000
21    Default  8988.000000       great  8988.000000  29.0000  29.0000
34    Default  5595.000000        good  5595.000000  28.0000  28.0000
17    Default  4942.000000       price  4942.000000  27.0000  27.0000
2075  Default  3098.000000      kindle  3098.000000  26.0000  26.0000
1856  Default  2164.000000       loves  2164.000000  25.0000  25.0000
2432  Default  6926.000000      tablet  6926.000000  24.0000  24.0000
164   Default  2050.000000     product  2050.000000  23.0000  23.0000
105   Default  1740.000000        year  1740.000000  22.0000  22.0000
520   Default  2114.000000         old  2114.000000  21.0000  21.0000
754   Default  2944.000000        easy  2944.000000  20.0000  20.0000
20    Default  1286.000000       brand  1286.000000  19.0000  19.0000
191   Default  2469.000000        kids  2469.000000  18.0000  18.0000
29    Default  2251.000000         buy  2251.000000  17.0000  17.0000
23    Default  2451.000000        long  2451.000000  16.0000  16.0000
651   Default  1713.000000       games  1713.000000  15.0000  15.0000
785   Default  1211.000000        read  1211.000000  14.0000  14.0000
945   Default  1210.000000     reading  1210.000000  13.0000  13.0000
386   Default  1020.000000       value  1020.000000  12.0000  12.0000
1267  Default   942.000000    daughter   942.000000  11.0000  11.0000
1678  Default  1589.000000      screen  1589.000000  10.0000  10.0000
9     Default  2265.000000        work  2265.000000   9.0000   9.0000
28    Default  3182.000000      bought  3182.000000   8.0000   8.0000
2086  Default  1519.000000       books  1519.000000   7.0000   7.0000
7     Default  1308.000000     quality  1308.000000   6.0000   6.0000
31    Default   854.000000   christmas   854.000000   5.0000   5.0000
90    Default  3489.000000        love  3489.000000   4.0000   4.0000
727   Default   959.000000         son   959.000000   3.0000   3.0000
74    Default   869.000000      brands   869.000000   2.0000   2.0000
431   Default   821.000000        deal   821.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
2526   Topic6    75.619965  connection   101.647644   1.9472  -6.0669
1019   Topic6   142.843597        went   236.085114   1.7406  -5.4309
1209   Topic6    79.955040        wait   111.703178   1.9087  -6.0112
316    Topic6   111.968307    keyboard   177.655426   1.7814  -5.6744
765    Topic6    97.512558      button   149.703735   1.8144  -5.8126
2075   Topic6   740.483826      kindle  3098.083740   0.8118  -3.7853
510    Topic6   141.430374         won   276.360352   1.5731  -5.4408
234    Topic6   312.302399      device  1035.709717   1.0442  -4.6487
424    Topic6   147.023483        home   311.225830   1.4931  -5.4020
237    Topic6   282.337189         new   989.040405   0.9894  -4.7495
28     Topic6   524.748413      bought  3182.682617   0.4405  -4.1297
221    Topic6   301.686951        best  1423.278809   0.6917  -4.6832
2785   Topic6   206.581497          hd   714.607788   1.0020  -5.0619
1675   Topic6   129.495255        wifi   284.485443   1.4560  -5.5290
29     Topic6   354.927460         buy  2251.546387   0.3956  -4.5207
252    Topic6   143.889267    expected   360.776794   1.3238  -5.4236
24     Topic6   397.538330      amazon  3860.358398  -0.0302  -4.4073
439    Topic6   139.824646     replace   359.173645   1.2996  -5.4522
2432   Topic6   439.06338

In [75]:
lda[corpus[0]]

[(0, 0.01524379),
 (1, 0.015174701),
 (2, 0.6496695),
 (3, 0.28946745),
 (4, 0.015277437),
 (5, 0.015167123)]

In [76]:
distro = [lda[d] for d in corpus]

def update(doc):
    d_dist = {k:0 for k in range(0,15)}
    for t in doc:
        d_dist[t[0]] = t[1]
    return d_dist

new_distro = [update(d) for d in distro]

In [ ]:
new_distro

In [77]:
df.head(1)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokens
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s..."


In [82]:
df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')

In [83]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.015244,0.015175,0.649771,0.289366,0.015277,0.015167,0,0,0,0,0,0,0,0,0
1,0.027965,0.027917,0.860523,0.027834,0.027924,0.027837,0,0,0,0,0,0,0,0,0
2,0.041763,0.041887,0.353154,0.479617,0.041748,0.041831,0,0,0,0,0,0,0,0,0
3,0.027838,0.027839,0.860424,0.028237,0.027877,0.027785,0,0,0,0,0,0,0,0,0
4,0.027809,0.027783,0.615493,0.273267,0.027829,0.027820,0,0,0,0,0,0,0,0,0


In [79]:
df = pd.DataFrame.from_records(new_distro)
df.columns= topics

ValueError: Length mismatch: Expected axis has 15 elements, new values have 6 elements

### What topics are people talking about in their amazon tech review?



## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling